In [1]:
import pandas as pd
import numpy as np
import Levenshtein
import re
import unidecode
import json

data = pd.read_csv('../data/Banco de Súmulas - Sumulas.csv')

# keep only pair classes
classes = ['470', '49er', '49erFX', 'Nacra 17']
data = data[data['Classe Vela'].isin(classes)]

C:\Users\Carol Erthal\AppData\Local\Temp\ipykernel_4000\3196152256.py:8: DtypeWarning: Columns (8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/Banco de Súmulas - Sumulas.csv')


In [3]:
# treating strings

# unidecode
data['Nome Competidor'] = data['Nome Competidor'].apply(unidecode.unidecode)

# replace more than one space by one space
data['Nome Competidor'] = data['Nome Competidor'].str.replace(r'\s+', ' ')

# substituir [;] por [virgula espaço]
data['Nome Competidor'] = data['Nome Competidor'].str.replace(r';', r', ')

# substituir [ / ] por [virgula espaço]
data['Nome Competidor'] = data['Nome Competidor'].str.replace(r' / ', r', ')

for index, row in data.iterrows():
    if re.match(r'^([A-Z]{2,})', row['Nome Competidor']):
        # ERTHAL Carol LAMARCA Felipe
        data.at[index, "Nome Competidor"] = re.sub(r'([a-z]{2,}) ([A-Z]{2,})', r'\1, \2', data.at[index, "Nome Competidor"])
                
        # ERTHAL CarolLAMARCA Felipe
        data.at[index, "Nome Competidor"] = re.sub(r'([a-z]{2,})([A-Z]{2,})', r'\1, \2', data.at[index, "Nome Competidor"])

    else:
        # Carol ERTHALFelipe LAMARCA
        data.at[index, "Nome Competidor"] = re.sub(r'([A-Z])([A-Z][a-z])', r'\1, \2', data.at[index, "Nome Competidor"])

        # Carol ERTHAL Felipe LAMARCA
        data.at[index, "Nome Competidor"] = re.sub(r'([A-Z]) ([A-Z][a-z])', r'\1, \2', data.at[index, "Nome Competidor"])

        # Carol ErthalFelipe Lamarca / Erthal CarolLamarca Felipe
        data.at[index, "Nome Competidor"] = re.sub(r'([a-z])([A-Z][a-z])', r'\1, \2', data.at[index, "Nome Competidor"])

# upper case
data['Nome Competidor'] = data['Nome Competidor'].str.upper()

C:\Users\Carol Erthal\AppData\Local\Temp\ipykernel_4000\3585964946.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Nome Competidor'] = data['Nome Competidor'].str.replace(r'\s+', ' ')


In [4]:
unique = list(data['Nome Competidor'].unique())

to_fix_manually = {}
for name in unique:
    # se tiver mais de uma virgula, adicionar como chave no dicionario
    if len(re.findall(r',', name)) > 1:
        to_fix_manually[name] = []
    # se nao tiver virgula, adicionar como chave no dicionario
    elif len(re.findall(r',', name)) == 0:
        to_fix_manually[name] = []

In [6]:
# to_fix_manually to json
with open('../data/to_fix_manually.json', 'w') as fp:
    json.dump(to_fix_manually, fp)

In [130]:
# # fix manually the ones found above

# # read json file as dict
# with open('manual_fix.json', 'r') as f:
#     manual_fix = json.load(f)

# # replace keys by values in data using manual_fix
# for key, value in manual_fix.items():
#     data['Nome Competidor'] = data['Nome Competidor'].str.replace(key, value)

FileNotFoundError: [Errno 2] No such file or directory: 'manual_fix.json'

In [7]:
# get unique names again
unique = list(data['Nome Competidor'].unique())

# fix mixed up names function
def are_names_same(str1, str2):
    # Split the strings into words, remove spaces, and sort the lists of words
    name1 = sorted(str1.split())
    name2 = sorted(str2.split())
    
    # Check if the sorted lists of words are equal
    return name1 == name2

# get pairs
pairs = []
for i in range(len(unique)):
    for j in range(i+1, len(unique)):
        if are_names_same(unique[i], unique[j]):
            pairs.append((unique[i], unique[j]))

# substitute pairs in data
for pair in pairs:
    data['Nome Competidor'] = data['Nome Competidor'].str.replace(pair[1], pair[0])

In [9]:
names = data['Nome Competidor'].unique()
# levenshtein distance

# create df where rows and columns are the names and the values are the levenshtein distance
names_comparison = np.array(np.meshgrid(names, names)).T.reshape(-1,2)

# create a third column with the levenshtein distance
names_comparison = pd.DataFrame(names_comparison, columns=['name', 'other_names'])
names_comparison['levenshtein_distance'] = names_comparison.apply(lambda row: Levenshtein.distance(row['name'], row['other_names']), axis=1)

In [10]:
ld1 = names_comparison[names_comparison['levenshtein_distance'] == 1]
ld2 = names_comparison[names_comparison['levenshtein_distance'] == 2]
ld3 = names_comparison[names_comparison['levenshtein_distance'] == 3]
ld4 = names_comparison[names_comparison['levenshtein_distance'] == 4]
ld5 = names_comparison[names_comparison['levenshtein_distance'] == 5]
ld6 = names_comparison[names_comparison['levenshtein_distance'] == 6]
ld7 = names_comparison[names_comparison['levenshtein_distance'] == 7]
ld8 = names_comparison[names_comparison['levenshtein_distance'] == 8]
ld9 = names_comparison[names_comparison['levenshtein_distance'] == 9]
ld10 = names_comparison[names_comparison['levenshtein_distance'] == 10]

In [11]:
# create dict with the names that are similar (dont insert same pair twice)
similar_names = {}

for ld in [ld1, ld2, ld3, ld4, ld5, ld6, ld7, ld8, ld9, ld10]:
    for _, row in ld.iterrows():
        # if the name is already in the dict
        if row['name'] in similar_names:
            # if the other name is not in the list, add it
            if row['other_names'] not in similar_names[row['name']]:
                similar_names[row['name']].append(row['other_names'])
            # if it is, do nothing
            else:
                pass

        # if the name is not in the dict and the other name is
        elif row['other_names'] in similar_names.values():
            # if the name isnt on other_name key, add it
            if row['name'] not in similar_names[row['other_names']]:
                similar_names[row['other_names']].append(row['name'])
            # if it is, do nothing
            else:
                pass
            
        # if none of the names are in the dict, add them as a new key
        else:
            # check which appears the most in the data and add that one as the key
            if data[data['Nome Competidor'] == row['name']].shape[0] > data[data['Nome Competidor'] == row['other_names']].shape[0]:
                similar_names[row['name']] = [row['other_names']]
            else:
                similar_names[row['other_names']] = [row['name']]

In [12]:
# for every unique name not in the dict as key or value, add it as a key with an empty list as value
for name in names:
    if name not in similar_names and name not in similar_names.values():
        similar_names[name] = []

In [13]:
# ordenar similar_names por ordem alfabetica das chaves
similar_names = {k: v for k, v in sorted(similar_names.items(), key=lambda item: item[0])}

In [14]:
# similar_names to json
with open('../data/similar_names.json', 'w') as fp:
    json.dump(similar_names, fp)